In [19]:
import requests
import chess.pgn
import chess
import json
from chess.engine import Cp

In [20]:
url = "https://explorer.lichess.ovh/lichess"

params = {
    "variant": "standard",
    "speeds": "blitz,rapid,classical",
    "ratings": "2200,2500",
    "fen": None
}

url_eval = "https://lichess.org/api/cloud-eval"

params_eval = {
    "variant": "standard",
    "multiPv": "1",
    "fen": None
}

In [21]:
game = chess.pgn.Game()

game.headers["Event"] = "Prova API Lichess"

In [22]:
def get_cloud_eval(node):
    
    params_eval['fen'] = node.board().fen()
    response = requests.get(url_eval, params=params_eval)
    tree = json.loads(response.content.decode())
        
    return tree['pvs'][0]['cp']

In [23]:
def get_comment(tree):
    # Commento dello score
    w = tree['white']
    b = tree['black']
    d = tree['draws']

    tot = w+b+d

    perc_w = (int)(w/tot * 100)
    perc_b = (int)(b/tot * 100)
    perc_d = (int)(d/tot * 100)

    comment = f"{perc_w}/{perc_d}/{perc_b}"
    
    return comment

In [24]:
def MakeMove(move, node, move_comment, max_depth):   
        
    # eseguo la mossa richiesta
    child_node = node.add_variation(chess.Move.from_uci(move))

    child_node.comment = move_comment
    
    eval = get_cloud_eval(child_node)
    
    child_node.set_eval(chess.engine.PovScore(Cp(42), chess.WHITE))
    
    # interrompo la ricerca se raggiungo la profondità massima
    if(child_node.ply() > max_depth):
        return
    
    # ottengo il codice fen della posizione
    params["fen"] = child_node.board().fen()
    
    # eseguo la chiamata all'API lichess
    response = requests.get(url, params=params)
    
    # parsing della response
    tree = json.loads(response.content.decode())
    
    comment = get_comment(tree)
    
    MakeMove(tree['moves'][0]['uci'], child_node, comment, max_depth)
    MakeMove(tree['moves'][1]['uci'], child_node, comment, max_depth)

In [25]:
MakeMove("e2e4", game, "", 2)

In [26]:
# Salvataggio della partita in formato PGN
with open("repertorio.pgn", "w") as f:
    exporter = chess.pgn.FileExporter(f)
    game.accept(exporter)

In [27]:
print(game)

[Event "Prova API Lichess"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "?"]
[Black "?"]
[Result "*"]

1. e4 { [%eval 0.42] } 1... c5 { 47/7/44 [%eval 0.42] } ( 1... e5 { 47/7/44 [%eval 0.42] } 2. Nf3 { 48/7/44 [%eval 0.42] } ( 2. Nc3 { 48/7/44 [%eval 0.42] } ) ) 2. Nf3 { 47/7/45 [%eval 0.42] } ( 2. Nc3 { 47/7/45 [%eval 0.42] } ) *
